# 🎤 Chatterbox TTS Server for Stockpile

**One-click setup** - Just run the cell below!

### Before running:
1. **Enable GPU**: Runtime → Change runtime type → **T4 GPU**
2. Click the play button below (or Shift+Enter)
3. Wait ~3 minutes for setup
4. Copy the URL and paste into Stockpile

---

In [ ]:
#@title 🚀 Run This Cell - One Click Setup
print("="*60)
print("🎤 CHATTERBOX TTS SERVER SETUP")
print("="*60)

# ============================================================
# STEP 1: Install Dependencies
# ============================================================
print("\n📦 Step 1/4: Installing packages...")
import subprocess
import sys

def run_cmd(cmd, desc):
    print(f"   {desc}")
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"   ⚠️ Warning: {result.stderr[:200] if result.stderr else 'Unknown error'}")
    return result.returncode == 0

# Install chatterbox from working fork
run_cmd('pip install -q "chatterbox-tts @ git+https://github.com/devnen/chatterbox.git@master"', "Installing Chatterbox TTS...")
run_cmd('pip install -q fastapi uvicorn python-multipart pyngrok', "Installing server dependencies...")

print("   ✅ Packages installed")

# ============================================================
# STEP 2: Load Model
# ============================================================
print("\n🧠 Step 2/4: Loading TTS model (this takes ~2 min)...")
import torch
from chatterbox.tts import ChatterboxTTS

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"   Device: {device.upper()}")

if device == "cpu":
    print("   ⚠️ WARNING: No GPU detected! Go to Runtime → Change runtime type → T4 GPU")

model = ChatterboxTTS.from_pretrained(device=device)
print("   ✅ Model loaded")

# ============================================================
# STEP 3: Create Server
# ============================================================
print("\n🖥️ Step 3/4: Creating API server...")

import base64
import io
import tempfile
import torchaudio
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import Response
from pydantic import BaseModel
from typing import Optional

app = FastAPI(title="Chatterbox TTS Server")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class TTSRequest(BaseModel):
    text: str
    exaggeration: float = 0.5
    cfg_weight: float = 0.5
    temperature: float = 0.8
    output_format: str = "mp3"
    voice_reference: Optional[str] = None

@app.get("/")
async def root():
    return {"status": "running", "model": "Chatterbox TTS"}

@app.get("/api/ui/initial-data")
async def health():
    return {"status": "ok", "device": device}

@app.post("/tts")
async def generate_tts(request: TTSRequest):
    try:
        text = request.text.strip()
        if not text:
            raise HTTPException(status_code=400, detail="Text required")
        
        print(f"Generating: {len(text)} chars")
        
        audio_prompt = None
        if request.voice_reference:
            try:
                audio_bytes = base64.b64decode(request.voice_reference)
                with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
                    f.write(audio_bytes)
                    audio_prompt = f.name
            except Exception as e:
                print(f"Voice ref error: {e}")
        
        wav = model.generate(
            text=text,
            audio_prompt_path=audio_prompt,
            exaggeration=request.exaggeration,
            cfg_weight=request.cfg_weight,
            temperature=request.temperature,
        )
        
        buffer = io.BytesIO()
        torchaudio.save(buffer, wav, model.sr, format="mp3")
        buffer.seek(0)
        
        print(f"Done: {buffer.getbuffer().nbytes} bytes")
        return Response(content=buffer.read(), media_type="audio/mpeg")
    
    except Exception as e:
        print(f"Error: {e}")
        raise HTTPException(status_code=500, detail=str(e))

print("   ✅ Server created")

# ============================================================
# STEP 4: Start Server + Tunnel
# ============================================================
print("\n🌐 Step 4/4: Starting server and tunnel...")

import threading
import time
import nest_asyncio
nest_asyncio.apply()

# Start uvicorn in background thread
def run_server():
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8004, log_level="warning")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(3)

# Setup ngrok tunnel (more reliable than cloudflared)
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Create tunnel
public_url = ngrok.connect(8004).public_url

print("\n" + "="*60)
print("🎉 TTS SERVER READY!")
print("="*60)
print(f"\n📋 Copy this URL into Stockpile:\n")
print(f"   {public_url}")
print(f"\n" + "="*60)
print("\n⚡ Features:")
print("   • No character limit - generate 10+ minute audio")
print("   • Voice cloning supported")
print(f"   • GPU accelerated ({device.upper()})")
print("\n⚠️  Keep this notebook running while using Stockpile!")
print("="*60)

# Keep alive
try:
    while True:
        time.sleep(60)
except KeyboardInterrupt:
    print("\nServer stopped.")